<a href="https://colab.research.google.com/github/projjal1/Machine-Learning-Sklearn/blob/main/Book_Price_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Hackathon 
https://www.machinehack.com/hackathons/predict_the_price_of_books/overview



In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
train_data=pd.read_excel('https://github.com/projjal1/datasets/blob/master/Book%20Price%20Pred%20Ds/Data_Train.xlsx?raw=true')
train_data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [ ]:
y=train_data['Price'].values
train_data=train_data.drop('Title',axis=1)
train_data=train_data.drop('Synopsis',axis=1)
train_data=train_data.drop('Edition',axis=1)
train_data=train_data.drop('Price',axis=1)

In [ ]:
print('Few labels: ',y[:5])

Few labels:  [220.   202.93 299.   180.   965.62]


In [ ]:
train_data.head()

,Author,Reviews,Ratings,Genre,BookCategory
0,Chris Kuzneski,4.0 out of 5 stars,8 customer reviews,Action & Adventure (Books),Action & Adventure
1,Arun Khopkar,3.9 out of 5 stars,14 customer reviews,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts"
2,Thomas Hobbes,4.8 out of 5 stars,6 customer reviews,International Relations,Humour
3,Agatha Christie,4.1 out of 5 stars,13 customer reviews,Contemporary Fiction (Books),"Crime, Thriller & Mystery"
4,Editors of Life,5.0 out of 5 stars,1 customer review,Photography Textbooks,"Arts, Film & Photography"


Let's create buckets

In [ ]:
import re

In [ ]:
def preprocess(data):
  for each in ['Author','Genre','BookCategory']:
    data[each]=pd.Categorical(data[each])
    data[each]=data[each].cat.codes
  reviews=[]
  for each in data['Reviews'].values:
    reviews.append(float(each[:3]))
  ratings=[]
  for each in data['Ratings'].values:
    res=re.search(r"\d*",each)
    ratings.append(float(res[0]))
  reviews=pd.DataFrame(reviews)
  ratings=pd.DataFrame(ratings)
  data['Ratings']=ratings
  data['Reviews']=reviews
  return data

In [ ]:
train=preprocess(train_data)
train.head()

,Author,Reviews,Ratings,Genre,BookCategory
0,615,4.0,8.0,1,0
1,307,3.9,14.0,74,2
2,3390,4.8,6.0,193,6
3,62,4.1,13.0,92,5
4,953,5.0,1.0,253,1


#Model training using sklearn Linear Regression module 
Let's compare the result of a tf NN and sklearn Linear Regression module

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train= scaler.fit_transform(train)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
model = Sequential()
model.add(Dense(8,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(x_train, y, batch_size=128,epochs=2000)

In [ ]:
#Submit part 
test_data=pd.read_excel('https://github.com/projjal1/datasets/blob/master/Book%20Price%20Pred%20Ds/Data_Test.xlsx?raw=true')
test_data=test_data.drop('Title',axis=1)
test_data=test_data.drop('Synopsis',axis=1)
test_data=test_data.drop('Edition',axis=1)

In [ ]:
submit=pd.read_excel('https://github.com/projjal1/datasets/blob/master/Book%20Price%20Pred%20Ds/Sample_Submission.xlsx?raw=true')

In [ ]:
test=preprocess(test_data)
x_test= scaler.fit_transform(test)

In [ ]:
res=[]
for itr in range(len(x_test)):
  dat=x_test[itr]
  dat=np.expand_dims(dat,axis=0)
  res.append(model.predict(dat)[0][0])
  print(itr)

In [ ]:
res=pd.DataFrame(res)
submit['Price']=res
submit.to_csv('final.csv',index=False)